In [89]:
import pandas as pd 
import numpy as np
from scipy.stats import randint
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import RandomizedSearchCV

In [56]:
boston = load_boston()

C:\Users\AHMED\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (

In [80]:
# Split the data into a training set and a testing set
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.3, random_state=42)

In [81]:
#Splitted Data
print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)


X_train shape is  (354, 13)
X_test shape is  (152, 13)
y_train shape is  (354,)
y_test shape is  (152,)


In [82]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(regressor, X_train, y_train, scoring ="neg_mean_squared_error",cv = 10)
tree_rmse_scores = np.sqrt(-scores)

In [83]:
print("Scores: ", tree_rmse_scores)
print("Mean: ", tree_rmse_scores.mean())
print("Standard Deviation: ", tree_rmse_scores.std())

Scores:  [4.38263619 3.81666667 7.71695464 6.30823976 4.08747215 4.75226262
 3.25900949 3.67415685 3.95344335 7.94367673]
Mean:  4.989451844574476
Standard Deviation:  1.6222558187493146


In [90]:
# Define the hyperparameter space
param_dist = {'max_depth': randint(1, 10),
              'min_samples_leaf': randint(1, 10),
              'min_samples_split': randint(2, 20)}

In [91]:
# Create a DecisionTreeRegressor object
dt = DecisionTreeRegressor()

In [92]:
# Create a RandomizedSearchCV object
rnd_search = RandomizedSearchCV(dt, param_distributions=param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error',random_state=42)

In [93]:
# Fit the RandomizedSearchCV object to the data
rnd_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(),
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017F4B5FA790>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017F4B676F10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017F4B5FAE80>},
                   random_state=42, scoring='neg_mean_squared_error')

In [94]:
# Print the best hyperparameters
print("Best Hyperparameters:", rnd_search.best_params_)

Best Hyperparameters: {'max_depth': 8, 'min_samples_leaf': 8, 'min_samples_split': 4}


In [95]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

5.080509913268099 {'max_depth': 7, 'min_samples_leaf': 4, 'min_samples_split': 16}
5.134392426506688 {'max_depth': 8, 'min_samples_leaf': 5, 'min_samples_split': 8}
5.253539560159928 {'max_depth': 3, 'min_samples_leaf': 7, 'min_samples_split': 12}
5.1354673071307415 {'max_depth': 8, 'min_samples_leaf': 5, 'min_samples_split': 5}
4.770987927717687 {'max_depth': 8, 'min_samples_leaf': 8, 'min_samples_split': 4}
5.2137577720270585 {'max_depth': 6, 'min_samples_leaf': 5, 'min_samples_split': 3}
4.952948323484786 {'max_depth': 8, 'min_samples_leaf': 6, 'min_samples_split': 3}
5.041067294280541 {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 13}
4.8289890477651065 {'max_depth': 6, 'min_samples_leaf': 9, 'min_samples_split': 18}
5.253539560159928 {'max_depth': 3, 'min_samples_leaf': 7, 'min_samples_split': 4}


In [98]:
final_model = rnd_search.best_estimator_
final_predictions = final_model.predict(X_test)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

3.9782675039718747

In [99]:
4.770987927717687 - 3.9782675039718747

0.7927204237458128